In [232]:
from IntegralQuery import SearchQuery, IntegralQuery, Filter, Range #################################################
import numpy as np
from dataclasses import dataclass
import astropy.units as u
from astropy.coordinates import SkyCoord
from datetime import datetime
from numba import njit, vectorize



@njit
def calculate_distance_matrix(quick_list, angle_weight, time_weight, max_distance):
    l = len(quick_list)
    distances = np.full((l,l), 2*max_distance)
    
    partitions = [0]
    for i in range(1,l):
        if quick_list[i,2]-quick_list[i-1,2] > max_distance/angle_weight:
            partitions.append(i)
    partitions.append(l)
    
    for i in range(len(partitions)-1):
        for j in range(partitions[i], partitions[i+1]):
            for k in range(j+1, partitions[i+1]):
                distances[j,k] = distances[k,j] = calculate_distance(quick_list[j],quick_list[k],
                                                                     angle_weight,time_weight)
                
    np.fill_diagonal(distances,0.)
            
    return partitions, distances

@njit
def calculate_distance(point1, point2, angle_weight, time_weight):
    ang_dis = np.arccos( np.sin(point1[1])*np.sin(point2[1]) # Large Error for very small angles
                        + np.cos(point1[1])*np.cos(point2[1]) * np.cos(point1[0] - point2[0]) )
    time_dis = abs(point1[2] - point2[2])
    return ( (angle_weight*ang_dis)**2 + (time_weight*time_dis)**2 )**0.5

@njit
def find_regions(distances, max_distance): ####################### Include Partitions?
    unconnected = [i for i in range(len(distances))]
    regions = []
    
    while not len(unconnected)==0:
        temp_region = [unconnected.pop(0)]
        search_index = 0
        while search_index < len(temp_region):
            l = len(unconnected)
            for i in range(l-1,-1,-1):
                if distances[ temp_region[search_index], unconnected[i] ] < max_distance:
                    temp_region.append(unconnected.pop(i))
            search_index += 1
        regions.append(temp_region)
        
    return regions
        
        
    
    


class Cluster:
    pass

@dataclass
class Pointing:
    '''
    Dataclass that represents a single Pointing
    '''
    scw_id: str
    sky_coords: SkyCoord
    start_time: datetime
    cluster: Cluster = None
    
    def distance_calculator(self, pointing2, angle_weight: float, time_weight: float): #####################################################
        return ( (angle_weight * self.sky_coords.separation(pointing2.sky_coords).deg)**2
                + (time_weight * abs( (self.start_time - pointing2.start_time).total_seconds()/86400 ) )**2 )**0.5






class ClusteredQuery:
    def __init__(self,
                 scw_ids, # Has to be sorted by START_DATE
                 angle_weight,
                 time_weight,
                 max_distance,
                 cluster_size_range = (3,5),
                 cluster_size_preferece_threshold = (2,2)):
        """
        Init the Clustered Query object. Used to cluster pointings
        
        """
        
        
        self._num_pointings = len(scw_ids)
        
        self.quick_list = np.zeros((self._num_pointings, 3))
        self.quick_list[:,0:2] = scw_ids[:,1:3]
        for i in range(self._num_pointings):
            self.quick_list[i,2] = (scw_ids[i,3] - datetime(2000,1,1,0,0,0)).total_seconds()/86400
            
        self.partitions, self.distance = calculate_distance_matrix(self.quick_list, angle_weight, time_weight, max_distance)
        
        
        self._pointings = np.array([Pointing(pointing[0],
                                       SkyCoord(pointing[1],pointing[2],frame="icrs",unit="deg"),
                                       pointing[3])
                              for pointing in scw_ids])
        
        self._distances = np.zeros((self._num_pointings, self._num_pointings))
        
        # for i in range(self._num_pointings):
        #     for j in range(i+1, self._num_pointings):
        #         self._distances[i,j] = self._distances[j,i] = self._pointings[i].distance_calculator(self._pointings[j], angle_weight, time_weight)
        
        
    @property
    def num_pointings(self):
        """
        :returns: Number of Pointings
        """
        return self._num_pointings
    
    @property
    def pointings(self):
        """
        :returns: Base List of Pointings
        """
        return self._pointings
    
    @property
    def distances(self):
        """
        :returns: Base Distance Matrix of Pointings
        """
        return self._distances
    







In [233]:
searchquerry = SearchQuery(object_name="Cyg X-1", resultmax=0)
cat = IntegralQuery(searchquerry)
f = Filter(SCW_TYPE="POINTING")
scw_ids = cat.apply_filter(f,True)

In [234]:
test = ClusteredQuery(scw_ids, 1., 1., 10.)

In [235]:
test.pointings

array([Pointing(scw_id='001130000050', sky_coords=<SkyCoord (ICRS): (ra, dec) in deg
           (299.543488, 35.301971)>, start_time=datetime.datetime(2002, 11, 16, 20, 6, 11), cluster=None),
       Pointing(scw_id='001130000070', sky_coords=<SkyCoord (ICRS): (ra, dec) in deg
           (299.590179, 35.201195)>, start_time=datetime.datetime(2002, 11, 16, 20, 18, 23), cluster=None),
       Pointing(scw_id='001130000080', sky_coords=<SkyCoord (ICRS): (ra, dec) in deg
           (299.590179, 35.201195)>, start_time=datetime.datetime(2002, 11, 16, 20, 55, 43), cluster=None),
       ...,
       Pointing(scw_id='250300240010', sky_coords=<SkyCoord (ICRS): (ra, dec) in deg
           (306.328033, 35.514473)>, start_time=datetime.datetime(2022, 5, 16, 13, 17, 35), cluster=None),
       Pointing(scw_id='250300250010', sky_coords=<SkyCoord (ICRS): (ra, dec) in deg
           (307.85379, 33.744278)>, start_time=datetime.datetime(2022, 5, 16, 14, 15, 5), cluster=None),
       Pointing(scw_id='2503

In [236]:
test.quick_list

array([[ 299.543488  ,   35.301971  , 1050.83762731],
       [ 299.590179  ,   35.201195  , 1050.84609954],
       [ 299.590179  ,   35.201195  , 1050.87202546],
       ...,
       [ 306.328033  ,   35.514473  , 8171.55387731],
       [ 307.85379   ,   33.744278  , 8171.59380787],
       [ 305.735168  ,   32.48539   , 8171.63373843]])

In [237]:
test.distance[0:10,770:780]

array([[42.64875815, 42.70224952, 42.7878189 , 42.77155362, 42.79569216,
        42.84393261, 42.8753621 , 42.86977125, 43.10530103, 43.14840649],
       [42.63738565, 42.69088279, 42.78198787, 42.75972509, 42.78659251,
        42.83143798, 42.86694702, 42.85780199, 43.09848836, 43.14159253],
       [42.61146921, 42.66496636, 42.75611051, 42.73381798, 42.76068543,
        42.80554186, 42.84105626, 42.83188945, 43.07258586, 43.11569   ],
       [42.60777955, 42.66095546, 42.66034825, 42.75620607, 42.70830223,
        42.81091715, 42.76584051, 42.84679913, 43.00230918, 43.04543676],
       [42.58642898, 42.63960472, 42.63898384, 42.73487617, 42.68693497,
        42.78958778, 42.74447535, 42.82546095, 42.98093686, 43.02406443],
       [42.52039343, 42.57387612, 42.61470036, 42.6218912 , 42.67099874,
        42.67405563, 42.72021416, 42.72879144, 42.98969254, 43.03278601],
       [42.5005703 , 42.55387378, 42.5795149 , 42.62111885, 42.62734607,
        42.67039836, 42.68041171, 42.71928964

In [238]:
p1 = np.array([300.,40.,0.])
p2 = np.array([300.,40.1,0.])
calculate_distance(p1,p2,1.,1.)

0.10000000000000056

In [239]:
calculate_distance_matrix(test.quick_list,1.,1.,10.)

([0,
  783,
  788,
  796,
  802,
  806,
  926,
  930,
  933,
  940,
  952,
  953,
  962,
  979,
  982,
  985,
  1226,
  1230,
  1233,
  1236,
  1480,
  1482,
  1490,
  1493,
  1496,
  1498,
  1501,
  1504,
  1509,
  1513,
  1516,
  1518,
  1521,
  1527,
  1533,
  1540,
  1545,
  1550,
  1609,
  1611,
  1617,
  1644,
  1703,
  1774,
  1924,
  1930,
  1936,
  1938,
  1939,
  2207,
  2209,
  2237,
  2397,
  2399,
  2691,
  2708,
  2723,
  2927,
  2936,
  2939,
  3089,
  3148,
  3207,
  3704,
  3706,
  3763,
  3822,
  4032,
  4194,
  4202,
  4260,
  4332,
  4442,
  4444,
  4448,
  4529,
  4683,
  4707,
  4715,
  4724,
  5008,
  5091,
  5134,
  5139,
  5240,
  5781,
  6174,
  6605,
  7441,
  7443,
  7544,
  7704,
  7758,
  7782,
  7786,
  7787,
  7813,
  7819,
  7834,
  7836,
  8142,
  8415,
  8422,
  8424,
  8428,
  8469,
  8471,
  8485,
  8933,
  8957,
  8963,
  8974,
  8976,
  8979,
  8983,
  9336,
  9356,
  9371,
  9384,
  9571,
  9676,
  9702],
 array([[ 0.        ,  0.10729474,  0.112

In [240]:
test.test=1

In [241]:
test.test

1

In [244]:
d = np.array([[0.,1.,1.,20.,20.,20.],
              [1.,0.,1.,20.,20.,3.],
              [1.,1.,0.,20.,20.,20.],
              [20.,20.,20.,0.,2.,20.],
              [20.,20.,20.,2.,0.,20.],
              [20.,3.,20.,20.,20.,20.]])

In [245]:
find_regions(d,10.)

[[0, 2, 1, 5], [3, 4]]